In [67]:
%use coroutines

import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.flow.MutableStateFlow
import kotlinx.coroutines.flow.StateFlow
import kotlinx.coroutines.flow.collect
import kotlinx.coroutines.GlobalScope
import kotlinx.coroutines.launch

In [2]:
import java.time.Instant

inline fun time(action: () -> Unit): Long {
    val start = Instant.now().toEpochMilli()
    action()
    return Instant.now().toEpochMilli() - start
}

suspend fun doWorld() = coroutineScope {  // this: CoroutineScope
    launch {
        delay(1010L)
        println("World!")
    }
    println("Hello")
}

fun main() = runBlocking {
    val t = time {
        doWorld()
        println("Lots of code")
    }
    println("Done $t ms")
}

main()

Hello
World!
Lots of code
Done 1033 ms


In [68]:
val _stateFlow = MutableStateFlow<Int>(0)
val stateFlow: StateFlow<Int> = _stateFlow

fun stateFlowEmitterValue() : Flow<Int> = flow {
    emit(1)
    emit(2)
    emit(3)
}

fun runTest() = GlobalScope.launch {
    stateFlowEmitterValue().collect {
        _stateFlow.emit(it)
    }
    
    stateFlow.collect {
        println("first: $it")
    }
    
    this.cancel()
}

runTest()

first: 3


StandaloneCoroutine{Active}@4e904fd5

In [69]:
fun runTest2() = GlobalScope.launch {
    _stateFlow.emit(5)
    
    stateFlow.collectLatest {
        println("second: $it")
    }
}

runTest2()

first: 5
second: 5


StandaloneCoroutine{Active}@4cbf4f53

In [70]:
fun stateFlowEmitterValue2() : Flow<Int> = flow {
    emit(6)
    emit(7)
    emit(8)
}

fun runTest3() = GlobalScope.launch {
    stateFlowEmitterValue2().collect {
        _stateFlow.emit(it)
    }
    
    stateFlow.collectLatest {
        println("third: $it")
    }
}

runTest3()

first: 8
second: 8
third: 8


StandaloneCoroutine{Active}@15dd5ac2

In [98]:
val _sharedFlow = MutableSharedFlow<Int>()
val sharedFlow = _sharedFlow.asSharedFlow()

fun sharedFlowmitterValue() : Flow<Int> = flow {
    emit(4)
    emit(5)
    emit(6)
}

fun runTest4() = GlobalScope.launch {
    sharedFlowmitterValue().collect {
        _sharedFlow.emit(it)
        println("emit: $it")
    }
    
    sharedFlow.collectLatest {
        println("first: $it")
    }
}

fun runTest5() = GlobalScope.launch {
    sharedFlow.collectLatest {
        println("second: $it")
    }
}

runTest4()
runTest5()

second: 4
emit: 4
second: 5
emit: 5
second: 6
emit: 6


StandaloneCoroutine{Active}@1b29d52b

In [94]:
fun runTest6() = GlobalScope.launch {
    sharedFlow.collectLatest {
        println("second: $it")
    }
}

runTest6()

StandaloneCoroutine{Active}@69d6a7cd